## Object-Oriented Design: War Card Game

Now let's design a card game. If you've never played War before (or if you have), 
here's the variant of the game that we will play. The
deck is split evenly between two players. On each turn, both players
reveal their first cards. The player showing the card with the highest rank takes both
cards and adds them to his or her deck and reshuffles his or her hand. If there is a tie, the players
reveal their next cards. If there is no longer a tie, the player with
the highest rank takes all four cards on the revealed stack into their hand (and reshuffles their hand). 
Otherwise, both players continue revealing their next cards until the tie is broken. 
The game continues until one player has collected all 52 cards.

What classes and methods do we need? Each card has a rank and a suit,
so it makes sense to make a **Card** class with these attributes. In our
game, "greater" cards are those with higher rank; cards with the same
rank are "equal", no matter what suit they come from.

A **Hand** is a collection of Cards, so we'll make that another class. We can give
and take cards from a Hand and also shuffle the cards in a Hand. We'll
want to know how many Cards are in a Hand, so we'll need a `num_cards` method,
among others.

A **Deck** is a special kind of Hand, populated with (in our case) the 
standard 52 cards. We can `deal` cards from a Deck.

A **Player** is also a Hand of cards, but with a (person) name also.

Finally, the **Game** class implements all of the game logic. This class
implements methods for dealing hands, taking a turn, and playing a
game to determine a winner.

### Card class

In [ ]:
class Card():
    def __init__(self, rank, suit):
        """ rank: integer from 2 to (including) 14
            suit: 'S' for Spades, or 'H' for Hearts, 
                  or 'C' for Clubs, or 'D' for Diamonds
        """
        assert suit in {'S', 'H', 'C', 'D'}
        assert rank in range(2,15)
        self.rank = rank
        self.suit = suit
        
    suit_str = {'S': "\u2660", 'H': "\u2661", 'C': "\u2663", 'D': "\u2662"}
    rank_str = {n: str(n) for n in range(2,11)}
    rank_str[11] = 'J'; rank_str[12] = 'Q'; rank_str[13] = 'K'; rank_str[14] = 'A' 
    
    def __str__(self): # '2♡'
        return f"{self.rank_str[self.rank]}{self.suit_str[self.suit]}"
    
    def __repr__(self): # Card(2,'H')
        return f"Card({self.rank},'{self.suit}')"
    
    def __gt__(self, other):
        return self.rank > other.rank
    
    def __lt__(self, other):
        return self.rank < other.rank
    
    def __eq__(self, other):
        return self.rank == other.rank

Creating Cards, and their repr and str representations:

In [ ]:
Card(2,'H') #new instance of Card

In [ ]:
repr(Card(2,'H'))

In [ ]:
str(Card(2,'H'))

Card comparisons -- based on rank, not suit:

In [ ]:
Card(2,'H') == Card(2,'S') #want True

In [ ]:
Card(2,'H') > Card(7,'S') #want False

In [ ]:
Card(2,'H') < Card(7,'S') #want True

Some customized notions of equality:

In [ ]:
Card(2,'H') is Card(2,'H') #want False; different instances

In [ ]:
Card(2,'H') == Card(2,'H') #want True; based on rank

In [ ]:
Card(2,'H') == Card(2,'S') #want True; based on rank

By implementing <, >, ==, we gain a powerful ability to use many Python built-in functions, like `max`, `min`, `sorted`, etc.!

In [ ]:
cards = [Card(3,'S'), Card(14,'D'), Card(10,'D'), Card(14,'H')]
print("cards:", cards) # note: str(<list>) creates a string of the list of <repr of list elements>
print("max:", max(cards))
print("min:", min(cards))
print("position of max card:", cards.index(max(cards)))
print("regular sorted:", [str(c) for c in sorted(cards)])
print("reverse sorted:", [str(c) for c in sorted(cards, reverse=True)])

### Hand class

In [ ]:
import random
class Hand():
    def __init__(self, cards=[]):
        self.cards = [] #avoid aliasing; receive_cards into new empty list
        self.receive_cards(cards)

    def receive_cards(self, cards):
        """ Receive cards into hand """
        self.cards.extend(cards)
    
    def receive_card(self, card):
        """ Receive single card into bottom of hand """
        self.receive_cards([card])
    
    def shuffle(self):
        """ Shuffle the deck by rearranging the cards in random order. """
        if self.cards:
            random.shuffle(self.cards)

    def give_card(self):
        """ Remove and return the card at the top of the hand. """
        assert self.num_cards > 0
        return self.cards.pop(0)
    
    def give_cards(self):
        """ Remove and yield cards from the top of the hand, until
            there are no more cards """
        while len(self.cards) > 0:
            yield self.cards.pop(0)
    
    @property
    def num_cards(self):
        return len(self.cards)

    def __iter__(self):
        """ Iterator over the cards in the hand. Does not remove
            the cards from the hand. """
        yield from self.cards

    def __repr__(self): # Hand([Card(2,'S')])
        return f"Hand({self.cards})"
    
    def __str__(self): # ['2♠']
        return str([str(card) for card in self.cards])

In [ ]:
h = Hand()
h.receive_card(Card(2,'S'))
h.receive_cards([Card(13,'H'), Card(7,'C')])
print("hand:", h)
print("num_cards:", h.num_cards) #instead of h.num_cards()
print("give card:", h.give_card())
print("rest of cards:", [str(c) for c in h.give_cards()])
h

### Deck class

In [ ]:
class Deck(Hand):
    def __init__(self):
        super().__init__(Deck.build_deck())
        self.size = self.num_cards
        self.shuffle()

    @staticmethod
    def build_deck():
        """
        Return a list of 52 cards, as in a standard deck.

        Suits are "H" (Hearts), "S" (Spades), "C" (Clubs), "D" (Diamonds).
        Ranks in order of increasing strength the numbered cards 
        2-10, 11 Jack, 12 Queen, 13 King, and 14 Ace.
        """
        suits = {"H", "S", "C", "D"}
        return [Card(rank, suit) for rank in range(2,15) for suit in suits]

In [ ]:
d = Deck()
print(d)
print("size of deck:", d.size)
print("give card:", d.give_card())
print("rest of cards:", [str(c) for c in d.give_cards()])

### Player class

In [ ]:
class Player(Hand):
    def __init__(self, name, cards=[]):
        super().__init__(cards)
        self.name = name
      
    def __repr__(self): # Player('Pam', [Card(10,'H')])
        return f"Player({repr(self.name)}, {self.cards})"
    
    def __str__(self):  # Player('Pam', ['10♡'])
        return f"<{repr(self.name)} has {[str(c) for c in self.cards]}>"

In [ ]:
p = Player('Pam')
p.receive_card(Card(10,'H'))
print(p)
p

### Game class

In [ ]:
class Game():
    def __init__(self, players):
        self.players = players
        self.deck = Deck()

    def deal(self):
        """ 
        Deal cards to both players. Each player takes one card at 
        a time from the deck.
        """
        while self.deck.num_cards > 0:
            for p in self.players:
                if self.deck.num_cards > 0:
                    p.receive_card(self.deck.give_card())
                else:
                    return
    
    def turn(self, do_print=False):
        """
        Get cards from both players. The player with the higher 
        rank takes all the cards in the pile.
        """
        assert len([p for p in self.players if p.num_cards > 0]) > 1

        prev_in_play = [p for p in self.players if p.num_cards > 0]
        last_cards = [p.give_card() for p in prev_in_play]
        in_play = [p for p in self.players if p.num_cards > 0]
        pile = last_cards
            
   
        # If there is a tie, get the next cards and add them to the
        # top (front of) the cards pile, if any players still have cards
        while len(in_play) > 1 and last_cards.count(max(last_cards)) > 1:
            if do_print: print("war:", [str(c) for c in pile], "TIE!")
            prev_in_play = in_play
            last_cards = [p.give_card() for p in self.players if p.num_cards > 0]
            in_play = [p for p in self.players if p.num_cards > 0]
            pile += last_cards
        
        if len(in_play) == 1:
            # There is a tie, but at least one player ran out of cards,
            # and that player loses.
            winner = in_play[0]
        else:
            winner = prev_in_play[last_cards.index(max(last_cards))]
        #winner gets pile, and shuffles hand
        winner.receive_cards(pile)
        winner.shuffle()

        
        if do_print: print("war:", [str(c) for c in pile], "=>", winner.name)

            
    def play(self, do_print=False):
        """
        Keep taking turns until a player has won (has all 52 cards).
        Return the winning player.
        """
        self.deal()
        if do_print: 
            for p in self.players: 
                print("at start", p)
                
        in_play = [p for p in self.players if p.num_cards > 0]
        while len(in_play) > 1:
            self.turn(do_print)
            in_play = [p for p in self.players if p.num_cards > 0]
    
        #decide and return winner (player object)
        p = in_play[0]
        if do_print: print(p.name + " wins!")

        return p
        
    def play_n_times(self, n, do_print=False):
        for p in self.players:
            p.wins = 0 #interesting! Add instance attribute on the fly

        for i in range(n):
            self.play(do_print).wins += 1

            # Return all cards players are holding back to the deck
            for p in self.players:
                self.deck.receive_cards(p.give_cards())
            self.deck.shuffle()
            self.deal()
        
        print("\nPlayed", n, "hands")
        for p in self.players:
            print("  ", p.name, "wins:", p.wins)

### Let's try it out...

In [ ]:
game = Game([Player("Amy"), Player("Brad")])
game.play(do_print = True)

## Many games
Let's extend so we can run many games and see who wins the most.

In [ ]:
game = Game([Player("Amy"), Player("Brad")])
game.play_n_times(3)
game.play_n_times(30)
game.play_n_times(100)

### Try with more than two players

In [ ]:
game = Game([Player("Amy"), Player("Brad"), Player("Carl"), Player("John")])
game.play_n_times(3)
game.play_n_times(30)
game.play_n_times(100)

Oops. We hard coded the game to only expect two players! We'll leave it as an exercise for the reader to go back and generalize `Game` to fix.